# Export 2018 Transactions and create xlsx-Files

In [24]:
import pandas as pd
import mysql.connector
from pyexcelerate import Workbook, Color, Style, Fill, Font

## Connect

In [2]:
db_connection = mysql.connector.connect(
    host="localhost",
    user="root",
    passwd="",
    database = "pharmagelder"
)

## Load Data

In [3]:
df_recipient = pd.read_sql("""SELECT 
      tra_fk_recipient as id,
      max(recipient.rec_name) as name,
      max(recipient.rec_address) as address,
      max(recipient.rec_location) as location,
      max(recipient.rec_plz) as plz,
      max(recipient.rec_type) as type,
      max(tra_year) as maxYear
    FROM transaction
    LEFT JOIN recipient ON rec_id = transaction.tra_fk_recipient
    WHERE tra_year = 2018
    GROUP BY tra_fk_recipient""", con=db_connection)

df_transaction = pd.read_sql("""SELECT 
    max(tra_fk_recipient) as id,
    max(tra_name_original) as name,
    max(tra_location_original) as location,
    max(tra_address_original) as address,
    max(tra_year) as maxYear,
    max(recipient.rec_type) as type,
    max(pha_name) as pharma,
    sum(tra_value) as value
  FROM transaction
  LEFT JOIN recipient ON rec_id = tra_fk_recipient
  LEFT JOIN pharma ON tra_fk_pharma = pha_id
  WHERE tra_year = 2018
  GROUP BY tra_name_original, tra_location_original, tra_address_original""", con=db_connection)

## Export

In [29]:
def df_to_sheet(name, df, wb):
    df = df.fillna("")

    values = [df.columns] + list(df.values)
    ws = wb.new_sheet(name, data=values)

    counter = 2
    for i, row in df.iterrows():
        if row['mainaddress']:
            ws.set_row_style(counter, Style(fill=Fill(background=Color(191, 191, 191, 0)), font=Font(bold=True)))
        counter += 1

In [31]:
df_recipient['mainaddress'] = True
df_transaction['mainaddress'] = False

#Concat
df_concat = pd.concat([df_recipient, df_transaction], sort=False)
df_concat = df_concat.sort_values(['id', 'mainaddress'], ascending = [True, False])

df_concat = df_concat.reset_index(drop=False)

df_hcp = df_concat[df_concat.type == 'hcp']
df_hco = df_concat[df_concat.type == 'hco']

df_hcp = df_hcp.set_index(['id', 'index'])
df_hco = df_hco.set_index(['id', 'index'])

#Write to Excel. Use pyexcelerate because of performance reasons
wb = Workbook()
df_to_sheet('hcp', df_hcp, wb)
df_to_sheet('hco', df_hco, wb)

wb.save('../../data/99. analyzes/excel/recipients/2018_transactions.xlsx')